In [1]:
pip install langchain langchain-text-splitters langchain-community bs4

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -U "langchain[google-genai]"

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install -qU langchain-google-genai

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install -qU langchain-chroma

Note: you may need to restart the kernel to use updated packages.


In [17]:
# RAG agent example (drop into a Jupyter notebook and run)
# Requirements (run in a cell first if not installed):
# !pip install langchain langchain-core langchain-openai langchain-text-splitters langchain-community bs4

import os
import getpass
import bs4
from typing import Tuple

# ------------------------------
# 1) Configure LangSmith tracing
# ------------------------------
# Option A: set environment variables in the notebook (recommended for interactive use)
# You will be prompted for your LangSmith API key and your OpenAI API key.
os.environ.setdefault("LANGSMITH_TRACING", "true")
os.environ.setdefault("LANGSMITH_API_KEY", "insert_key")
os.environ.setdefault("LANGSMITH_ENDPOINT", "https://eu.api.smith.langchain.com")
os.environ.setdefault("GOOGLE_API_KEY", "insert_key")
os.environ.setdefault("LANGSMITH_PROJECT", "rag_test")
os.environ.setdefault("USER_AGENT", "kth_gpt")

if not os.environ.get("LANGSMITH_API_KEY"):
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter LANGSMITH_API_KEY (will not echo): ")

# Also set your model provider keys (example: OpenAI)
if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter GOOGLE_API_KEY (will not echo): ")

# With LANGSMITH_TRACING="true" and LANGSMITH_API_KEY set the LangChain SDK will emit traces to LangSmith.
# Traces become visible in https://smith.langchain.com under your account.


# ------------------------------
# 2) Imports for LangChain pieces
# ------------------------------
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Google Gemini chat model integration
from langchain.chat_models import init_chat_model

# Google Gemini embeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# Chroma vector store
from langchain_chroma import Chroma

# Agent creation + tools
from langchain.agents import create_agent
from langchain.tools import tool

# Middleware for two-step RAG example
from langchain.agents.middleware import dynamic_prompt, ModelRequest

# ------------------------------
# 3) Select chat model + embeddings (Google Gemini)
# ------------------------------
# Initialize chat model via helper. Replace model name with one available to your account if needed.
model = init_chat_model("google_genai:gemini-2.5-flash-lite")

# Embeddings (Gemini embeddings)
embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

# ------------------------------
# 4) Load and split documents
# ------------------------------
# Example: Lilian Weng blog article from the provided docs
#bs4_strainer = bs4.SoupStrainer(class_=("mw-content-ltr mw-parser-output"))
loader = WebBaseLoader(
    web_paths=("https://www.aftonbladet.se/sportbladet/bloggar/f1bloggen/a/M7wMEm/f1-verstappen-spelar-psykspel-infor-finalen-i-abu-dhabi",),
    #bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()
assert len(docs) >= 1

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True,
)
all_splits = text_splitter.split_documents(docs)
print(f"Loaded {len(docs)} document(s), split into {len(all_splits)} chunks.")

# ------------------------------
# 5) Create Chroma vector store and index
# ------------------------------
vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # change or remove if you don't want persistence
)
doc_ids = vector_store.add_documents(documents=all_splits)
print("Indexed document chunk ids (sample):", doc_ids[:3])

# ------------------------------
# 6) Tool that wraps the vector store
# ------------------------------
@tool(response_format="content_and_artifact")
def retrieve_context(query: str) -> Tuple[str, list]:
    """
    Returns (serialized_text, retrieved_documents)
    - serialized_text: a string that will be shown to the model
    - retrieved_documents: the raw Document objects are attached as artifacts
    """
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content[:1000]}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

tools = [retrieve_context]

# ------------------------------
# 7) Build a simple agent (agentic RAG)
# ------------------------------
system_prompt = (
    "You have access to a tool that retrieves context from a website. "
    "Use the tool to help answer user queries. If you need supporting evidence, call the tool."
)
agent = create_agent(model, tools, system_prompt=system_prompt)

# Example: run an agentic RAG query (the agent may call the retrieve_context tool)
query = "Who is Max Verstappen?"
print("\n--- Agentic RAG run (streamed) ---\n")
for event in agent.stream({"messages": [{"role": "user", "content": query}]}, stream_mode="values"):
    event["messages"][-1].pretty_print()

# ------------------------------
# 8) Two-step RAG (single LLM call): use dynamic_prompt middleware
# ------------------------------
@dynamic_prompt
def prompt_with_context(request: ModelRequest) -> str:
    """Inject retrieved context based on the last user message into the system prompt."""
    last_query = request.state["messages"][-1].text
    retrieved_docs = vector_store.similarity_search(last_query, k=2)
    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)
    system_message = (
        "You are a helpful assistant. Use the following context in your response:\n\n"
        f"{docs_content}\n\n"
        "Answer the user's question using ONLY the context above where relevant; if no answer is available, say you don't know."
    )
    return system_message

agent_two_step = create_agent(model, tools=[], middleware=[prompt_with_context])

query2 = "What did Verstappen say in the interview"
print("\n--- Two-step RAG run (single call, streamed) ---\n")
for event in agent_two_step.stream({"messages": [{"role": "user", "content": query2}]}, stream_mode="values"):
    event["messages"][-1].pretty_print()

# ------------------------------
# 9) LangSmith tracing notes
# ------------------------------
# - With LANGSMITH_TRACING="true" and LANGSMITH_API_KEY set, LangChain will emit traces to LangSmith automatically.
# - Each run (tool calls, model calls, messages, artifacts) will be available in your LangSmith workspace:
#   https://smith.langchain.com
# - If traces do not appear: verify LANGSMITH_API_KEY, LANGSMITH_TRACING, and that your environment has network access.

Loaded 1 document(s), split into 11 chunks.
Indexed document chunk ids (sample): ['2b9e8b49-6cda-45a5-8b54-f5cc0fe6d092', '94512ac3-bd6a-4f4b-ba3f-43bd3b3c370d', 'b8002435-3996-45d4-86c2-d7728112c73b']

--- Agentic RAG run (streamed) ---

================================ Human Message =================================

Who is Max Verstappen?
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (b3b30aeb-7dc5-4b61-8e53-3e825aaad24c)
 Call ID: b3b30aeb-7dc5-4b61-8e53-3e825aaad24c
  Args:
    query: Who is Max Verstappen?
================================= Tool Message =================================
Name: retrieve_context

Source: {'source': 'https://www.aftonbladet.se/sportbladet/bloggar/f1bloggen/a/M7wMEm/f1-verstappen-spelar-psykspel-infor-finalen-i-abu-dhabi', 'title': 'F1: ”Verstappen spelar psykspel inför finalen i Abu Dhabi”', 'description': 'Max Verstappen har siktet inställt. På sin femte VM-titel. För att nå den behöve